In [121]:
import warnings
warnings.filterwarnings("ignore")
import tqdm
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [123]:
# Extract data from the pdf:
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [36]:
# Extracted data:
extracted_data = load_pdf("data/")

In [4]:
# Create text chunks:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    print(vars(text_chunks[0]))
    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print("Length of my chunk: ", len(text_chunks))

{'page_content': 'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', 'metadata': {'source': 'data\\Medicine_Encyclopedia.pdf', 'page': 0}, 'type': 'Document'}
Length of my chunk:  6983


In [6]:
# Download embeddings model:
def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [122]:
embeddings = download_hf_embeddings()

In [8]:
#embeddings

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length: ", len(query_result))

Length:  384


In [10]:
# Initializing the chromadb:
chromadb = Chroma.from_documents(
            documents=text_chunks,
            embedding=embeddings,
            persist_directory='./Data/chromadb')

In [11]:
query = "What are Allergies"
docs = chromadb.similarity_search(query, k=2)
print("Result", docs)

Result [Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.', metadata={'page': 659, 'source': 'data\\Medicine_Encyclopedia.pdf'}), Document(page_content='Although environmental medicine is gaining more\nrespect within conventional medicine, detoxificationKEY TERMS\nAllergen —A foreign substance, such as mites in\nhouse dust or animal dander, that when\ninhaled,causes the airways to narrow and pro-\nduces symptoms of asthma.\nAntibody —A protein, also called immunoglobu-\nlin, produced by immune system cells to remove\nantigens (the foreign subst

In [81]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Only return the helpful answer below and nothing else.

Context: {context}
Question: {question}

Helpful answer:
"""

In [82]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [83]:
# Callbacks support token-wise streaming:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [110]:
# Importing the model:
llm = LlamaCpp(
    model_path="./Model/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.4,
    max_tokens=2048,
    callback_manager=callback_manager,
    verbose=False)

In [111]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=chromadb.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt: ")
    if user_input.lower() == "exit":
        print("Chatbot: Thanks for your time!")
        break
    print("Response: ")
    result = qa({"query": user_input})